In [1]:
from os import walk
from tqdm import tqdm
from PIL import Image
import torch
from tqdm import tqdm
import glob
import numpy as np

In [12]:
path = "../construction_images/data/pretrain/images/"

files = glob.glob(path + "*.jpg")

In [13]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, device='cuda:0', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/kit/stud/utpqw/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-11-12 Python-3.8.6 torch-1.7.1 CUDA:0 (Tesla V100-SXM2-32GB, 32511MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [14]:
images = []
for file in tqdm(files):
    image = Image.open(file)
    images.append(image)

100%|██████████| 33344/33344 [00:05<00:00, 5712.66it/s]


In [15]:
batch_size = 200
batches = np.arange(0, len(images), batch_size)

In [16]:
results = []
for x in tqdm(range(len(batches) - 1)):
    results += model(images[batches[x]:batches[x+1]], size=80).pandas().xyxy

100%|██████████| 166/166 [02:58<00:00,  1.07s/it]


In [28]:
labels = {}
for x in range(len(results)):
    name = files[x].split('/')[-1]
    labels[name] = {}
    labels[name]['labels'] = []
    for row in results[x].iterrows():
        if row[-1]['confidence'] > 0.3:
            labels[name]['labels'].append(row[-1]['name'])

In [29]:
categories = set()

for image in labels.keys():
    for label in labels[image]['labels']:
        categories.add(label)

In [30]:
categories

{'airplane',
 'apple',
 'banana',
 'baseball bat',
 'bed',
 'bench',
 'bicycle',
 'bird',
 'boat',
 'book',
 'bottle',
 'bowl',
 'bus',
 'cake',
 'car',
 'cat',
 'cell phone',
 'chair',
 'clock',
 'couch',
 'cow',
 'cup',
 'dining table',
 'donut',
 'fire hydrant',
 'fork',
 'horse',
 'hot dog',
 'keyboard',
 'kite',
 'laptop',
 'microwave',
 'motorcycle',
 'mouse',
 'oven',
 'parking meter',
 'person',
 'potted plant',
 'refrigerator',
 'remote',
 'scissors',
 'sheep',
 'sink',
 'skateboard',
 'skis',
 'snowboard',
 'stop sign',
 'suitcase',
 'surfboard',
 'teddy bear',
 'tennis racket',
 'tie',
 'toilet',
 'toothbrush',
 'traffic light',
 'train',
 'truck',
 'tv',
 'umbrella',
 'vase'}

In [31]:
import json

with open('labels.json', 'w') as fp:
    json.dump(labels, fp)

In [34]:
categories = list(categories)
tasks = []

for category in categories: 
    task = {}
    task['label'] = category
    task['images-positive'] = []    
    task['images-negative'] = []
    
    for image in labels.keys():
        if category in labels[image]['labels']:
            task['images-positive'].append(image)
        else:
            if len(task['images-negative']) < 30:
                task['images-negative'].append(image)
                
    tasks.append(task)

In [35]:
with open('tasks.json', 'w') as fp:
    json.dump(tasks, fp)
tasks

[{'label': 'mouse',
  'images-positive': ['31541.jpg', '03870.jpg'],
  'images-negative': ['14337.jpg',
   '15529.jpg',
   '18266.jpg',
   '07425.jpg',
   '26957.jpg',
   '05648.jpg',
   '32252.jpg',
   '23088.jpg',
   '12131.jpg',
   '13794.jpg',
   '03344.jpg',
   '06848.jpg',
   '30328.jpg',
   '15810.jpg',
   '28164.jpg',
   '20304.jpg',
   '29008.jpg',
   '07788.jpg',
   '25865.jpg',
   '30623.jpg',
   '09853.jpg',
   '08668.jpg',
   '33514.jpg',
   '21007.jpg',
   '32328.jpg',
   '05021.jpg',
   '04617.jpg',
   '21296.jpg',
   '03469.jpg',
   '05284.jpg']},
 {'label': 'tie',
  'images-positive': ['14941.jpg',
   '18018.jpg',
   '03027.jpg',
   '27048.jpg',
   '26477.jpg',
   '23636.jpg',
   '14931.jpg'],
  'images-negative': ['14337.jpg',
   '15529.jpg',
   '18266.jpg',
   '07425.jpg',
   '26957.jpg',
   '05648.jpg',
   '32252.jpg',
   '23088.jpg',
   '12131.jpg',
   '13794.jpg',
   '03344.jpg',
   '06848.jpg',
   '30328.jpg',
   '15810.jpg',
   '28164.jpg',
   '20304.jpg',
   '2